<a href="https://colab.research.google.com/github/Zrmikstri/CS115-YOLOv3/blob/main/YOLOonCustomDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HUẤN LUYỆN MÔ HÌNH YOLOV3 TRÊN MỘT BỘ DATASET TÙY CHỈNH

# Step 1: Sao chép và xây dựng darknet

In [ ]:
# Clone darknet repository
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# Verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# Make darknet (build)
!make

# Step 1: Thu thập và gán nhãn dữ liệu
* Sử dụng Google's Open Images Dataset

Link: https://storage.googleapis.com/openimages/web/index.html

* Điều chỉnh lại nhãn dữ liệu lấy từ Google's Open Images Dataset sao cho phù hợp với YOLOv3, tham khảo link github: 
https://github.com/theAIGuysCode/OIDv4_ToolKit

# Step 2: Tùy chỉnh setting  file để huấn luyện

1.   Tạo file yolov3_custom.cfg với nội dung giống như file yolov3.cfg, sau đó:
    *   Thay đổi dòng batch thành batch=64
    *  Thay đổi dòng subdivisions thành subdivisions=64

    ![](https://i.imgur.com/tNmOckD.png)
    *   thay đổi dòng max_batches thành max_batches = number of classes * 2000
    *   Thay đổi dòng steps thành  steps=80% của  max_batches, 90% của max_batches
    
    ![](https://i.imgur.com/xprdFTB.png)

    *   Chỉnh classes=5 (đây là số class trong custom dataset)
    *   Thay [filters=255] thành filters = (classes + 5)* 3 tại dòng 603, dòng 689 và dòng 776

    ![](https://i.imgur.com/BnwJqZI.png)
2.   Tạo file obj.names, obj.data trong đường dẫn darknet\data

    ![](https://i.imgur.com/uNPoehG.png)

    ![](https://i.imgur.com/7h1C1xU.png)






4. Tạo một  file train.txt trong đường dẫn darknet\data chứa tất cả đường dẫn đến ảnh, mỗi đường dẫn một dòng
    ![](https://i.imgur.com/mMMQSlU.png)


5. Tất cả ảnh dùng để huấn luyện sẽ được đặt trên đường dẫn darknet\data\obj

In [ ]:
# Create train.txt
import os
image_files = []
os.chdir(os.path.join("data", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

# Step 3: Tải về pretrained weights cho lớp convolutional và đặt trong đường dẫn darknet\

In [ ]:
# Getting pretrained convolutional layer weights
!wget http://pjreddie.com/media/files/darknet53.conv.74

# Step 4: Thực hiện huấn luyện



# Bắt đầu huấn luyện lần đầu tiên
```
!./darknet detector train <path to obj.data> <path to custom config> darknet53.conv.74 -dont_show
```



In [ ]:
# train your custom detector
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show


## Tiếp tục huấn luyện nếu có dừng giữa chừng trong quá trình huấn luyện
```
!./darknet detector train <path to obj.data> <path to custom config> <path to last weights> 
```



In [ ]:
# Continue trainning
## Go to directory darknet
import os
os.chdir('/content/drive/MyDrive/ĐỒ ÁN MÔN HỌC CS115/YOLO on custom dataset/darknet')
!chmod +x ./darknet

!./darknet detector train data/obj.data cfg/yolov3_custom.cfg backup/yolov3_custom_last.weights -map -dont_show

# Step 5: Thực hiện kiểm tra mô hình trên dữ liệu mới
Sau khi hoàn thành quá trình huấn luyện, chuyển setting của fie yolov3_custom.cfg để kiểm tra mô hình

In [ ]:
# Setting file yolov3_custom.cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
%cd ..

In [ ]:
# Testing
import os
os.chdir('/content/drive/MyDrive/ĐỒ ÁN MÔN HỌC CS115/YOLO on custom dataset/darknet')
!./darknet detector test data/obj.data cfg/yolov3_custom.cfg /content/drive/MyDrive/YOLO/darknet/backup/yolov3_custom_final.weights /content/drive/MyDrive/YOLO/Data/strawberry.jpg

In [ ]:
# define functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image, (3 * width, 3 * height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
imShow('/content/drive/MyDrive/YOLO/darknet/predictions.jpg')